# Variance Threshold

This is a component that removes all low-variance features using an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters and model hyperparameters
Components may declare (and use) these default parameters:
- dataset
- target

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/). <br />
You may also declare custom parameters to set when running an experiment.

Select the hyperparameters and their respective values to be used when training the model:
- threshold

These parameters are just a few offered by the model class, you may also use another existing parameter. <br />
Check the [model parameters](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold) for more information.

In [ ]:
# parameters
dataset = "boston" #@param {type:"string"}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# hyperparameters
threshold = 0.3 #@param {type:"number", label:"Limiar", description:"Atributos com variância menor que o limiar serão removidos."}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Fit a model using sklearn.feature_selection.VarianceThreshold

In [ ]:
import pandas as pd
from platiagro.featuretypes import NUMERICAL
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

# selects the indexes of numerical features
numerical_indexes = (featuretypes == NUMERICAL)

pipeline = Pipeline(steps=[
    ('estimator', VarianceThreshold(threshold=threshold))
])

X_thr = pipeline.fit_transform(X[:, numerical_indexes])
X = np.concatenate((X[:, ~numerical_indexes], X_thr), axis=1)

# apply the indexes to get the new column sequence
columns = np.concatenate((columns[~numerical_indexes], 
                          columns[numerical_indexes][pipeline.named_steps.estimator.get_support()]))

# Put data back in a pandas.DataFrame
df = pd.DataFrame(data=X, columns=columns)
df[target] = pd.Series(y)

## Save dataset

Stores the transformed dataset in a object storage.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline,
           columns=columns.tolist(),
           numerical_indexes=numerical_indexes)